# Small_Scale_After_Annotations.ipynb
This notebook:
* Combines the outputs from `english-fisher-annotations` across all the transcript versions.
* Saves as csv.

In [1]:
import os
import difflib
import json
import string
import re

import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
df_small_scale = pd.read_csv("./csv/small_scale_texts.csv", index_col=False)
df_GroundTruth_max = pd.read_csv("./csv/df-english-fisher-annotations-test-GroundTruth-max.csv", index_col=False)
df_GroundTruth_min = pd.read_csv("./csv/df-english-fisher-annotations-test-GroundTruth-min.csv", index_col=False)
df_GroundTruth_neutral = pd.read_csv("./csv/df-english-fisher-annotations-test-GroundTruth-neutral.csv", index_col=False)
df_GoogleASR = pd.read_csv("./csv/df-english-fisher-annotations-test-GoogleASR.csv", index_col=False)
df_WhisperX = pd.read_csv("./csv/df-english-fisher-annotations-test-WhisperX.csv", index_col=False)

# keep all cols in the merge
mdf = None
mdf = pd.merge(df_small_scale, df_GroundTruth_max[["show_filename_prefix", "GroundTruth-max_parse", "GroundTruth-max_orig_dys", "GroundTruth-max_dys"]], on="show_filename_prefix")
mdf = pd.merge(mdf, df_GroundTruth_min[["show_filename_prefix", "GroundTruth-min_parse", "GroundTruth-min_orig_dys", "GroundTruth-min_dys"]], on="show_filename_prefix")
mdf = pd.merge(mdf, df_GroundTruth_neutral[["show_filename_prefix", "GroundTruth-neutral_parse", "GroundTruth-neutral_orig_dys", "GroundTruth-neutral_dys"]], on="show_filename_prefix")
mdf = pd.merge(mdf, df_GoogleASR[["show_filename_prefix", "GoogleASR_parse", "GoogleASR_orig_dys", "GoogleASR_dys"]], on="show_filename_prefix")
mdf = pd.merge(mdf, df_WhisperX[["show_filename_prefix", "WhisperX_parse", "WhisperX_orig_dys", "WhisperX_dys"]], on="show_filename_prefix")
mdf.index += 1
display(mdf)

# save df
mdf.to_csv("./csv/df-english-fisher-annotations-test-all3.csv", header=True)

# for index, row in mdf.iterrows():
#     print("GROUND-TRUTH:", row["GroundTruth-punctuated"])
#     print("GOOGLE:", row["GoogleASR"])
#     print("WHISPER:", row["WhisperX"])
#     print()
#     print()

,Unnamed: 0,nums,ids,labels,percent_diff,show_uri,show_name,show_description,publisher,language,rss_link,episode_uri,episode_name,episode_description,duration,show_filename_prefix,episode_filename_prefix,GroundTruth,GroundTruth-max,GroundTruth-neutral,GroundTruth-min,GoogleASR,WhisperX,rpl_GroundTruth,rpl_GoogleASR,rpl_WhisperX,GroundTruth-max_parse,GroundTruth-max_orig_dys,GroundTruth-max_dys,GroundTruth-min_parse,GroundTruth-min_orig_dys,GroundTruth-min_dys,GroundTruth-neutral_parse,GroundTruth-neutral_orig_dys,GroundTruth-neutral_dys,GoogleASR_parse,GoogleASR_orig_dys,GoogleASR_dys,WhisperX_parse,WhisperX_orig_dys,WhisperX_dys
1,1,1,1ll5WGWjWANfGKHZfrpL5A,not-scripted,14.20,spotify:show:5HLsz7WFjW8hzJurMDdozi,Granger Smith Podcast,"American, Texan, father, husband, musician, Gr...",Granger Smith,['en'],https://anchor.fm/s/11170a28/podcast/rss,spotify:episode:1ll5WGWjWANfGKHZfrpL5A,How do you move on from THIS?,"Episode 21: On this week's episode, I answered...",25.891600,show_5HLsz7WFjW8hzJurMDdozi,1ll5WGWjWANfGKHZfrpL5A,if you're ever in deep deep grief and you wond...,"If you're ever in deep, deep grief, and you wo...","If you're ever in deep, deep grief and you won...","If you're ever in deep, deep grief and you won...",If you're ever in deep deep grief and you wond...,"If you're ever in deep, deep grief and you won...",if you're ever in deep deep grief and you wond...,if you're ever in deep deep grief and you wond...,if you're ever in deep deep grief and you wond...,(S (SBAR (UNK if) (S (S (NP (UNK you)) (VP (UN...,if _ you _ 're _ ever _ in _ deep E deep _ gri...,if _ you _ 're _ ever _ in _ deep E deep _ gri...,(S (SBAR (UNK if) (S (S (NP (UNK you)) (VP (UN...,if _ you _ 're _ ever _ in _ deep E deep _ gri...,if _ you _ 're _ ever _ in _ deep E deep _ gri...,(S (SBAR (UNK if) (S (S (NP (UNK you)) (VP (UN...,if _ you _ 're _ ever _ in _ deep E deep _ gri...,if _ you _ 're _ ever _ in _ deep E deep _ gri...,(S (SBAR (UNK if) (S (S (NP (UNK you)) (VP (UN...,if _ you _ 're _ ever _ in _ deep E deep _ gri...,if _ you _ 're _ ever _ in _ deep E deep _ gri...,(S (SBAR (UNK if) (S (S (NP (UNK you)) (VP (UN...,if _ you _ 're _ ever _ in _ deep E deep _ gri...,if _ you _ 're _ ever _ in _ deep E deep _ gri...
2,2,2,1wHRMsWVurmo56xd1AcSfe,scripted,1.24,spotify:show:1g056e2x0Y9AwW6CQF3qA5,Mythology,Myths endure for a reason. This episodic audio...,Parcast Network,['en'],https://feeds.megaphone.fm/mythology,spotify:episode:1wHRMsWVurmo56xd1AcSfe,Orphan Boy & Elk Dog Pt. 1,Long Arrow is an orphan and an outcast—unable ...,36.144767,show_1g056e2x0Y9AwW6CQF3qA5,1wHRMsWVurmo56xd1AcSfe,something to note all myths have many versions...,Something to note: all myths have many version...,Something to note: all myths have many version...,Something to note: all myths have many version...,Something to note all myths have many versions...,"Something to note, all myths have many version...",something to note all myths have many versions...,something to note all myths have many versions...,something to note all myths have many versions...,(NP (NP (UNK something)) (SBAR (S (VP (UNK to)...,something _ to _ note _ all _ myths _ have _ m...,something _ to _ note _ all _ myths _ have _ m...,(NP (NP (UNK something)) (SBAR (S (VP (UNK to)...,something _ to _ note _ all _ myths _ have _ m...,something _ to _ note _ all _ myths _ have _ m...,(NP (NP (UNK something)) (SBAR (S (VP (UNK to)...,something _ to _ note _ all _ myths _ have _ m...,something _ to _ note _ all _ myths _ have _ m...,(NP (NP (UNK something)) (SBAR (S (VP (UNK to)...,something _ to _ note _ all _ myths _ have _ m...,something _ to _ note _ all _ myths _ have _ m...,(NP (NP (UNK something)) (SBAR (S (VP (UNK to)...,something _ to _ note _ all _ myths _ have _ m...,something _ to _ note _ all _ myths _ have _ m...
3,3,3,3i4qZ6FqzXdj8UiMeuaX3q,not-scripted,13.80,spotify:show:2z1LtWVnflRUONFAo0FADb,Gamecock Central Podcast Network,The Gamecock Central Podcast Network brings yo...,Gamecoc